In [1]:
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.tsa.seasonal import STL


In [2]:
read_dataset = pd.read_csv("data/swm_trialA_1K.csv", sep=";")

# reverse dataset because datetimes are reversed
data = read_dataset.iloc[::-1, ::-1]
sampled_user = data.sample(1).iloc[0]["user key"]



In [3]:
sampled_user_data = data[:][data["user key"] == sampled_user]
sampled_user_data['datetime'] = pd.to_datetime(sampled_user_data['datetime'] , format="%d/%m/%Y %H:%M:%S")

sampled_user_data.drop("user key", axis=1, inplace=True)

In [4]:
fixed_data = pd.DataFrame(sampled_user_data)

start = fixed_data['datetime'].min()
end = fixed_data['datetime'].max()
fixed_data.drop_duplicates(subset=['datetime'], keep='first', inplace=True)

all_dates = pd.date_range(start=start, end=end, freq='1h')
all_dates_df = pd.DataFrame({'datetime': all_dates})
all_dates_df.set_index('datetime', inplace=True)
fixed_data.set_index('datetime', inplace=True)
combined_df = all_dates_df.reindex(fixed_data.index.union(all_dates_df.index))

combined_df = combined_df.join(fixed_data, how='left')
combined_df['predicted'] = np.where(combined_df['meter reading'].notna(), 'no', 'yes')


In [5]:
# round to nearest minute and get rid of duplicated timestamps

combined_df.index = combined_df.index.round('h')

combined_df = combined_df[~combined_df.index.duplicated(keep='first')]

In [6]:
model_auto = auto_arima(combined_df['meter reading'].dropna(),seasonal=True,
                         stepwise=True, trace=True)
p, d, q = model_auto.order

Performing stepwise search to minimize aic
 ARIMA(2,2,2)(0,0,0)[0]             : AIC=910.430, Time=2.34 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=927.177, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=920.534, Time=0.01 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.23 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=911.592, Time=0.11 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=906.480, Time=0.13 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(2,2,0)(0,0,0)[0]             : AIC=917.867, Time=0.02 sec
 ARIMA(3,2,1)(0,0,0)[0]             : AIC=891.792, Time=0.23 sec
 ARIMA(3,2,0)(0,0,0)[0]             : AIC=902.213, Time=0.04 sec
 ARIMA(4,2,1)(0,0,0)[0]             : AIC=inf, Time=0.31 sec
 ARIMA(3,2,2)(0,0,0)[0]             : AIC=901.295, Time=0.11 sec
 ARIMA(4,2,0)(0,0,0)[0]             : AIC=903.196, Time=0.06 sec
 ARIMA(4,2,2)(0,0,0)[0]             : AIC=903.241, Time=0.42 sec
 ARIMA(3,2,1)(0,0,0)[0] intercept   : AIC=906.314, Time=0.0

In [7]:
model = ARIMA(combined_df['meter reading'], order=(p, d, q))  
model_fit = model.fit()

forecast = model_fit.predict(start=combined_df.index.min(), end=combined_df.index.max())
print(forecast)
combined_df['meter reading'] = combined_df['meter reading'].fillna(forecast)
combined_df['meter reading'] = combined_df['meter reading'].astype(int)

combined_df['diff'] = combined_df['meter reading'].diff()


C:\Users\Octavian Floare\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
C:\Users\Octavian Floare\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
C:\Users\Octavian Floare\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)


2015-09-22 00:00:00         0.000000
2015-09-22 01:00:00     84045.091387
2015-09-22 02:00:00     56066.600541
2015-09-22 03:00:00     56047.191520
2015-09-22 04:00:00     56072.913003
                           ...      
2017-05-19 20:00:00    228680.782765
2017-05-19 21:00:00    228691.776964
2017-05-19 22:00:00    228702.771162
2017-05-19 23:00:00    228713.765360
2017-05-20 00:00:00    228724.759558
Freq: h, Name: predicted_mean, Length: 14545, dtype: float64


In [8]:
# when i generate a value with arima and then i have a reading i usually get a negative diff so i have to just repeat the reading
for i in range(1, len(combined_df.index)):
    # if i find a negative diff
    if combined_df.loc[combined_df.index[i], 'diff'] < 0:
        # look for the previous meter reading
        prev_v = combined_df.iloc[i-1]['meter reading']
        combined_df.at[combined_df.index[i], 'meter reading'] = prev_v

# recompute the diff column
combined_df['diff'] = combined_df['meter reading'].diff()



In [9]:
combined_df.index = pd.to_datetime(combined_df.index, errors='coerce')

In [10]:
conditions = [
    (combined_df.index.hour >= 5) & (combined_df.index.hour < 12),  # Morning: 5 AM - 11:59 AM
    (combined_df.index.hour >= 12) & (combined_df.index.hour < 17), # Afternoon: 12 PM - 4:59 PM
    (combined_df.index.hour >= 17) & (combined_df.index.hour < 21), # Evening: 5 PM - 8:59 PM
    (combined_df.index.hour >= 21) | (combined_df.index.hour< 5)   # Night: 9 PM - 4:59 AM
]

time_of_day = ['Morning', 'Afternoon', 'Evening', 'Night']

combined_df['time of day'] = np.select(conditions, time_of_day, default=np.nan)


In [11]:
predicted = (combined_df["predicted"] == 'yes')

In [12]:
stl = STL(combined_df['meter reading'], seasonal=25) 

result = stl.fit()

trend = result.trend
seasonal = result.seasonal
residual = result.resid

plt.figure(figsize=(10, 8)) 
plt.subplot(4, 1, 1)
plt.plot(combined_df.index, combined_df['meter reading'], label='Original')
plt.legend(loc='upper left')
plt.title('Original Series')

plt.subplot(4, 1, 2)
plt.plot(combined_df.index, trend, label='Trend', color='orange')
plt.legend(loc='upper left')
plt.title('Trend Component')

plt.subplot(4, 1, 3)
plt.plot(combined_df.index, seasonal, label='Seasonal', color='green')
plt.legend(loc='upper left')
plt.title('Seasonal Component')

plt.subplot(4, 1, 4)
plt.plot(combined_df.index, residual, label='Residual', color='red')
plt.legend(loc='upper left')
plt.title('Residual Component')

plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
residual_mean = residual.mean()
residual_std = residual.std()
threshold = 3 * residual_std
anomalies = combined_df[abs(residual) > threshold]
print("Anomalies Detected:")
print(anomalies)
plt.figure(figsize=(12, 6))
plt.plot(combined_df.index, combined_df['meter reading'], label='Original Data')
plt.scatter(anomalies.index, anomalies['meter reading'], color='red', label='Anomalies')
plt.legend(loc='upper left')
plt.title('Detected Anomalies')
plt.show()

In [ ]:
#display(HTML(combined_df.to_html()))

In [ ]:
df = combined_df.copy().drop(columns = ['predicted', 'time of day'])

hourly_consumption = df.resample('h').sum()
hourly_pattern = hourly_consumption.groupby(hourly_consumption.index.hour).mean()
daily_pattern = df.groupby(df.index.dayofweek).mean()
monthly_pattern = df.groupby(df.index.month).mean()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(hourly_pattern.index, hourly_pattern['diff'], marker='o')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Consumption')
plt.title('Average Water Consumption by Hour of the Day')
plt.grid(True)
plt.xticks(range(0, 24))
plt.show()


In [ ]:
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

plt.figure(figsize=(12, 6))
plt.bar(days_of_week, daily_pattern['diff'])
plt.xlabel('Day of the Week')
plt.ylabel('Average Consumption')
plt.title('Average Water Consumption by Day of the Week')
plt.show()


In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

plt.figure(figsize=(12, 6))
plt.bar(months, monthly_pattern['diff'])
plt.xlabel('Month')
plt.ylabel('Average Consumption')
plt.title('Average Water Consumption by Month')
plt.show()


In [ ]:
# Resample to daily data
daily_consumption = combined_df.resample('D').sum()

# Identify peak days
peak_days = daily_consumption.nlargest(10, 'diff')


plt.figure(figsize=(12, 6))
plt.plot(daily_consumption.index, daily_consumption['diff'], label='Daily Consumption')
plt.scatter(peak_days.index, peak_days['diff'], color='red', label='Peak Days')
plt.xlabel('Date')
plt.ylabel('Total Daily Consumption')
plt.title('Daily Water Consumption with Peak Days Highlighted')
plt.legend()
plt.show()
